In [ ]:
import torch
import time
import os
import matplotlib.pyplot as plt
# Benchmarking setup
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
start_time = time.time()

# Memory, time, and efficiency tracking
memory_usage_history = []
time_per_iteration = []

# Output directory for sample renderings
output_dir = "Precisionresults/render_samples"
os.makedirs(output_dir, exist_ok=True)

# Move all relevant variables to the correct device.
renderer_grid = renderer_grid.to(device)
renderer_mc = renderer_mc.to(device)
target_cameras = target_cameras.to(device)
target_images = target_images.to(device)
target_silhouettes = target_silhouettes.to(device)

# Set the seed for reproducibility
torch.manual_seed(1)

# Instantiate the radiance field model.
neural_radiance_field = NeuralRadianceField().to(device)

# Instantiate the Adam optimizer. We set its master learning rate to 1e-3.
lr = 1e-3
optimizer = torch.optim.Adam(neural_radiance_field.parameters(), lr=lr)

# Set up mixed precision training for speed
scaler = torch.cuda.amp.GradScaler()  # For mixed precision

# Batch size (consider increasing based on GPU memory) bruker en rtx 3070 8GB 115W notebook
batch_size = 6  # shall be at 6 for notebook

# Number of iterations
n_iter = 200

# Loss history buffers
loss_history_color, loss_history_sil = [], []

# The main optimization loop
for iteration in range(n_iter):
    iter_start_time = time.time()

    if iteration == round(n_iter * 0.90):
        print('Decreasing LR 10-fold ...')
        optimizer = torch.optim.Adam(
            neural_radiance_field.parameters(), lr=lr * 0.1
        )
    
    # Zero the optimizer gradient
    optimizer.zero_grad()
    
    # Sample random batch indices
    batch_idx = torch.randperm(len(target_cameras))[:batch_size]
    
    # Sample the minibatch of cameras
    batch_cameras = FoVPerspectiveCameras(
        R = target_cameras.R[batch_idx], 
        T = target_cameras.T[batch_idx], 
        znear = target_cameras.znear[batch_idx],
        zfar = target_cameras.zfar[batch_idx],
        aspect_ratio = target_cameras.aspect_ratio[batch_idx],
        fov = target_cameras.fov[batch_idx],
        device = device,
    )
    
    # Evaluate the nerf model
    with torch.cuda.amp.autocast():  # Mixed precision context
        rendered_images_silhouettes, sampled_rays = renderer_mc(
            cameras=batch_cameras, 
            volumetric_function=neural_radiance_field
        )
        rendered_images, rendered_silhouettes = rendered_images_silhouettes.split([3, 1], dim=-1)
        
        # Compute silhouette error
        silhouettes_at_rays = sample_images_at_mc_locs(
            target_silhouettes[batch_idx, ..., None], 
            sampled_rays.xys
        )
        sil_err = huber(rendered_silhouettes, silhouettes_at_rays).abs().mean()

        # Compute color error
        colors_at_rays = sample_images_at_mc_locs(
            target_images[batch_idx], 
            sampled_rays.xys
        )
        color_err = huber(rendered_images, colors_at_rays).abs().mean()

        # Total loss
        loss = color_err + sil_err
    
    # Log the loss history
    loss_history_color.append(float(color_err))
    loss_history_sil.append(float(sil_err))
    
    # Log memory usage
    current_memory_usage = torch.cuda.memory_allocated() / (1024 ** 2)  # in MB
    max_memory_usage = torch.cuda.max_memory_allocated() / (1024 ** 2)  # in MB
    memory_usage_history.append((current_memory_usage, max_memory_usage))


    if iteration % 10 == 0:
        # Compute the average time per iteration only if there are entries in time_per_iteration
        if len(time_per_iteration) >= 10:
            avg_time = sum(time_per_iteration[-10:]) / 10
        elif len(time_per_iteration) > 0:
            avg_time = sum(time_per_iteration) / len(time_per_iteration)
        else:
            avg_time = 0.0  # Default value for the first iteration

        print(f"Iteration {iteration:05d}: loss color = {float(color_err):1.2e}, "
              f"loss silhouette = {float(sil_err):1.2e}, "
              f"Current memory = {current_memory_usage:.2f} MB, "
              f"Max memory = {max_memory_usage:.2f} MB, "
              f"Avg time per iteration = {avg_time:.2f}s")


    # Backpropagate and update the model using mixed precision
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    iter_end_time = time.time()
    time_per_iteration.append(iter_end_time - iter_start_time)

    # Visualize and save sample renderings every 500 iterations
    if iteration % 1000 == 0:
        show_idx = torch.randperm(len(target_cameras))[:1]
        sample_render = show_full_render(
            neural_radiance_field,
            FoVPerspectiveCameras(
                R = target_cameras.R[show_idx], 
                T = target_cameras.T[show_idx], 
                znear = target_cameras.znear[show_idx],
                zfar = target_cameras.zfar[show_idx],
                aspect_ratio = target_cameras.aspect_ratio[show_idx],
                fov = target_cameras.fov[show_idx],
                device = device,
            ), 
            target_images[show_idx][0],
            target_silhouettes[show_idx][0],
            loss_history_color,
            loss_history_sil,
        )
        plt.savefig(os.path.join(output_dir, f"render_iter_{iteration}.png"))

    # Reset peak memory stats for the next iteration
    torch.cuda.reset_peak_memory_stats()

# Final timing
end_time = time.time()
print(f"Total training time: {end_time - start_time:.2f} seconds")

# Post-training benchmarks



# Average time per iteration
avg_iteration_time = sum(time_per_iteration) / len(time_per_iteration)
print(f"Average time per iteration: {avg_iteration_time:.2f}s")
print(f"Total training time: {sum(time_per_iteration):.2f}s")

# Plot loss history for quality assessment
def plot_loss_history():
    plt.figure(figsize=(10, 5))
    plt.plot(loss_history_color, label="Color Loss")
    plt.plot(loss_history_sil, label="Silhouette Loss")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.title(f"Loss History {n_iter} Iterations" )
    plt.legend()
    plt.savefig(os.path.join('Precisionresults', f"loss_history.png"))
    plt.show()

plot_loss_history()

# **Place the plot_loss_history function here** (after the training loop ends)
def plot_loss_historylast_N(last_n=100):
    # Ensure the number of iterations is not less than the desired range
    start_iter = max(0, len(loss_history_color) - last_n)
    
    # Plotting the last `last_n` iterations of the loss history
    plt.figure(figsize=(10, 5))
    plt.plot(loss_history_color[start_iter:], label="Color Loss")
    plt.plot(loss_history_sil[start_iter:], label="Silhouette Loss")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.title(f"Loss History (Last {last_n} Iterations)")
    plt.legend()
    plt.savefig(os.path.join('Precisionresults', f"loss_history_last_{last_n}_iterations.png"))
    plt.show()

# Call the plot_loss_history function to visualize the last 2000 iterations' loss history
plot_loss_historylast_N(last_n=100)  # You can change last_n if you want a different number of iterations


In [ ]:
import torch
import time
import os
import matplotlib.pyplot as plt

# Benchmarking setup
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
start_time = time.time()

# Memory, time, and efficiency tracking
memory_usage_history = []
time_per_iteration = []

# Output directory for sample renderings
output_dir = "Precisionresults/render_samples"
os.makedirs(output_dir, exist_ok=True)

# Move all relevant variables to the correct device.
renderer_grid = renderer_grid.to(device)
renderer_mc = renderer_mc.to(device)
target_cameras = target_cameras.to(device)
target_images = target_images.to(device)
target_silhouettes = target_silhouettes.to(device)

# Set the seed for reproducibility
torch.manual_seed(1)

# Instantiate the radiance field model.
neural_radiance_field = NeuralRadianceField().to(device)

# Instantiate the Adam optimizer. We set its master learning rate to 1e-3.
lr = 1e-3
optimizer = torch.optim.Adam(neural_radiance_field.parameters(), lr=lr)

# Set up mixed precision training for speed
scaler = torch.cuda.amp.GradScaler()  # For mixed precision

# Batch size (consider increasing based on GPU memory) bruker en rtx 3070 8GB 115W notebook
batch_size = 6  # shall be at 6 for notebook

# Number of iterations
n_iter = 10000

# Loss history buffers
loss_history_color, loss_history_sil = [], []

# The main optimization loop
for iteration in range(n_iter):
    iter_start_time = time.time()

    if iteration == round(n_iter * 0.90):
        print('Decreasing LR 10-fold ...')
        optimizer = torch.optim.Adam(
            neural_radiance_field.parameters(), lr=lr * 0.1
        )
    
    # Zero the optimizer gradient
    optimizer.zero_grad()
    
    # Sample random batch indices
    batch_idx = torch.randperm(len(target_cameras))[:batch_size]
    
    # Sample the minibatch of cameras
    batch_cameras = FoVPerspectiveCameras(
        R = target_cameras.R[batch_idx], 
        T = target_cameras.T[batch_idx], 
        znear = target_cameras.znear[batch_idx],
        zfar = target_cameras.zfar[batch_idx],
        aspect_ratio = target_cameras.aspect_ratio[batch_idx],
        fov = target_cameras.fov[batch_idx],
        device = device,
    )
    
    # Evaluate the nerf model
    with torch.cuda.amp.autocast():  # Mixed precision context
        rendered_images_silhouettes, sampled_rays = renderer_mc(
            cameras=batch_cameras, 
            volumetric_function=neural_radiance_field
        )
        rendered_images, rendered_silhouettes = rendered_images_silhouettes.split([3, 1], dim=-1)
        
        # Compute silhouette error
        silhouettes_at_rays = sample_images_at_mc_locs(
            target_silhouettes[batch_idx, ..., None], 
            sampled_rays.xys
        )
        sil_err = huber(rendered_silhouettes, silhouettes_at_rays).abs().mean()

        # Compute color error
        colors_at_rays = sample_images_at_mc_locs(
            target_images[batch_idx], 
            sampled_rays.xys
        )
        color_err = huber(rendered_images, colors_at_rays).abs().mean()

        # Total loss
        loss = color_err + sil_err
    
    # Log the loss history
    loss_history_color.append(float(color_err))
    loss_history_sil.append(float(sil_err))
    
    # Log memory usage
    current_memory_usage = torch.cuda.memory_allocated() / (1024 ** 2)  # in MB
    max_memory_usage = torch.cuda.max_memory_allocated() / (1024 ** 2)  # in MB
    memory_usage_history.append((current_memory_usage, max_memory_usage))


    if iteration % 10 == 0:
        # Compute the average time per iteration only if there are entries in time_per_iteration
        if len(time_per_iteration) >= 10:
            avg_time = sum(time_per_iteration[-10:]) / 10
        elif len(time_per_iteration) > 0:
            avg_time = sum(time_per_iteration) / len(time_per_iteration)
        else:
            avg_time = 0.0  # Default value for the first iteration

        print(f"Iteration {iteration:05d}: loss color = {float(color_err):1.2e}, "
              f"loss silhouette = {float(sil_err):1.2e}, "
              f"Current memory = {current_memory_usage:.2f} MB, "
              f"Max memory = {max_memory_usage:.2f} MB, "
              f"Avg time per iteration = {avg_time:.2f}s")


    # Backpropagate and update the model using mixed precision
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    iter_end_time = time.time()
    time_per_iteration.append(iter_end_time - iter_start_time)

    # Visualize and save sample renderings every 500 iterations
    if iteration % 1000 == 0:
        show_idx = torch.randperm(len(target_cameras))[:1]
        sample_render = show_full_render(
            neural_radiance_field,
            FoVPerspectiveCameras(
                R = target_cameras.R[show_idx], 
                T = target_cameras.T[show_idx], 
                znear = target_cameras.znear[show_idx],
                zfar = target_cameras.zfar[show_idx],
                aspect_ratio = target_cameras.aspect_ratio[show_idx],
                fov = target_cameras.fov[show_idx],
                device = device,
            ), 
            target_images[show_idx][0],
            target_silhouettes[show_idx][0],
            loss_history_color,
            loss_history_sil,
        )
        plt.savefig(os.path.join(output_dir, f"render_iter_{iteration}.png"))

    # Reset peak memory stats for the next iteration
    torch.cuda.reset_peak_memory_stats()

# Final timing
end_time = time.time()
print(f"Total training time: {end_time - start_time:.2f} seconds")

# Post-training benchmarks

# Average time per iteration
avg_iteration_time = sum(time_per_iteration) / len(time_per_iteration)
print(f"Average time per iteration: {avg_iteration_time:.2f}s")
print(f"Total training time: {sum(time_per_iteration):.2f}s")


# **Place the plot_loss_history function here** (after the training loop ends)
def plot_loss_history(last_n=2000):
    # Ensure the number of iterations is not less than the desired range
    start_iter = max(0, len(loss_history_color) - last_n)
    
    # Plotting the last `last_n` iterations of the loss history
    plt.figure(figsize=(10, 5))
    plt.plot(loss_history_color[start_iter:], label="Color Loss")
    plt.plot(loss_history_sil[start_iter:], label="Silhouette Loss")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.title(f"Loss History (Last {last_n} Iterations)")
    plt.legend()
    plt.savefig(os.path.join('Precisionresults', f"loss_history_last_{last_n}_iterations.png"))
    plt.show()

# Call the plot_loss_history function to visualize the last 2000 iterations' loss history
plot_loss_history(last_n=2000)  # You can change last_n if you want a different number of iterations

